In [ ]:
keep = {"t1", "t2", "t3"}

for name in list(globals().keys()):
    if name not in keep and not name.startswith("_"):
        del globals()[name]

In [1]:
%%time
#DATA IMPORTATION -- and general model formulation
from __future__ import division
from pyomo.environ import *

from pyomo.opt import SolverStatus, TerminationCondition
import pandas
import pandas as pd
import time
import numpy
import pickle
import random
import sqlite3
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from matplotlib.ticker import MaxNLocator
from scipy.stats import gaussian_kde

path_data = "/projappl/project_2011004/BarkBeetle/"
path_out = "/projappl/project_2011004/BarkBeetle/opt_data/"

CPU times: user 1.21 s, sys: 241 ms, total: 1.45 s
Wall time: 14.3 s


In [ ]:
%%time
class optimization_stochastic:
    def __init__(self):
        c = 0
        self.area = 901.238
        self.data1 = pd.read_parquet(path_data + 'barkbeetle_data_v2.parquet', engine='pyarrow')
        self.data_orig = pd.read_parquet(path_data + 'barkbeetle_data_v2.parquet', engine='pyarrow')
        self.data_orig = self.data_orig.set_index(['stand','schedule','sample','period'])
        #To reduce the data a bit
        self.combinations = 100
        self.data1 = self.data1[self.data1['sample']<=self.combinations]
        self.data = self.data1
        
        data1 = self.data.set_index(['stand','schedule','sample','period'])
        stands = list(data1.loc[slice(None),slice(None),0,1].index.get_level_values(0))
        self.data = self.data[self.data['stand'].isin(stands)]


        self.data_opt = self.data

        self.all_data = self.data_opt

        self.Index_values = self.all_data.set_index(['stand','schedule']).index.unique()
        self.Index_sample_year = self.all_data.set_index(['sample','period']).index.unique()
        self.all_data = self.all_data.set_index(['stand','schedule','sample','period'])
        
        self.all_data = self.all_data.fillna(0)
        
        self.createModel()
        self.ADD_OBJECTIVE_FUNCTION()

    def ADD_OBJECTIVE_FUNCTION(self):
        self.model1.NPV= Var(within=NonNegativeReals)
        def NPV_INVENTORY(model1):
            row_sum = sum(sum( (self.all_data.npv.loc[(s,r,sam,12.0)]*self.model1.X1[(s,r)]*self.area)  for (s,r) in self.model1.index1) for sam in range(0,self.combinations))/self.combinations
            return self.model1.NPV ==row_sum
        self.model1.NPV_INV= Constraint(rule=NPV_INVENTORY)

        self.model1.Periodic_volume= Var(self.model1.year,self.model1.sample,within=NonNegativeReals)
        def PERIOD_INVENTORY(model1,p,sam):
            row_sum = sum( (self.all_data.vol_t.loc[(s,r,sam,p)]*self.model1.X1[(s,r)]*self.area)  for (s,r) in self.model1.index1)
            return self.model1.Periodic_volume[p,sam] ==row_sum
        self.model1.Period_INV= Constraint(self.model1.year,self.model1.sample,rule=PERIOD_INVENTORY)

        # A deterministic approach to ensure even flow -- useful to calculate the maximium even flow (a max the minimum problem)
        self.model1.DEC_inc = Var(self.model1.sample,within=NonNegativeReals)
        # INC constraints
        def inc_bounded_rule(model1, sam,p):
            INC2 = sum(
                (self.all_data.vol_t.loc[(s,r,sam,p)] ) * self.model1.X1[(s, r)] *self.area
                for (s,r) in self.model1.index1)
            return INC2 >= self.model1.DEC_inc[sam]

        self.model1.INC_bounded = Constraint(self.Index_sample_year, rule=inc_bounded_rule)


        def outcome_rule(model1):
            NPV = self.model1.NPV
            return NPV
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        #CVAR Part:
        ##CVAR: construct CVAR model for stochastic model

        #Downside CVaR
        self.model1.CVAR_down = Var(self.model1.year,within=Reals, initialize=1)
        self.model1.VAR_down = Var(self.model1.year,within=Reals, initialize=1)
        self.model1.posVAR_down = Var(self.model1.year,self.model1.sample, within=NonNegativeReals, initialize=1)
        self.model1.negVAR_down = Var(self.model1.year,self.model1.sample, within=NonNegativeReals, initialize=1)
        self.model1.mod_L_plus_down = Var(self.model1.year,self.model1.sample, within=NonNegativeReals, initialize=1)
        self.model1.mod_L_neg_down = Var(self.model1.year,self.model1.sample, within=NonNegativeReals, initialize=1)
        self.model1.alpha_risk_down = Param(default=0.80, mutable=True)
        self.model1.min_CVaR_down = Param(default=0, mutable=True)
        self.model1.max_CVaR_down = Param(default=1, mutable=True)
        self.model1.target_down = Param(default=3000, mutable=True)
        self.model1.CVAR_down_Tot = Var(within=Reals)
        self.model1.samples = len(self.model1.sample)

        #Downside CVaR Constraints LOG
        def CVAR_constraint_down(model1,k):
            CVAR_down = self.model1.VAR_down[k] + (1/((1-self.model1.alpha_risk_down)*self.model1.samples))*sum(self.model1.posVAR_down[k,sample] for sample in self.model1.sample)
            return self.model1.CVAR_down[k] == CVAR_down
        self.model1.CVAR_constraint_down = Constraint(self.model1.year,rule=CVAR_constraint_down)

        def MOD_L_P_constraint_down(model1,k,it):
            VAL_down = self.model1.mod_L_plus_down[k,it] -self.model1.VAR_down[k]+self.model1.negVAR_down[k,it]-self.model1.posVAR_down[k,it]
            return VAL_down == 0
        self.model1.MOD_L_P_down = Constraint(self.model1.year,self.model1.sample,rule=MOD_L_P_constraint_down)

        def MOD_TARGET_constraint_down(model1,k,it):
            row_sum = sum(((self.all_data.vol_t.loc[(s,r,it,k)])* self.model1.X1[(s,r)]*self.area) for (s,r) in self.model1.index1)
            VAL = self.model1.target_down-row_sum - self.model1.mod_L_plus_down[k,it]+self.model1.mod_L_neg_down[k,it]
            return VAL == 0
        self.model1.MOD_Target_down = Constraint(self.model1.year,self.model1.sample,rule=MOD_TARGET_constraint_down)

        def CVAR_down_constraint(model1):
            row_sum = sum(self.model1.CVAR_down[k] for k in self.model1.year)
            return self.model1.CVAR_down_Tot == row_sum 
        self.model1.CVAR_tot_down = Constraint(rule=CVAR_down_constraint)

                                    
    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        self.model1 = ConcreteModel()
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0))))
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(3))))
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.sample = Set(initialize = list(set(self.all_data.index.get_level_values(2))))
        self.model1.Index_values = self.Index_values

        
        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: #stand_set
                index.append((s,r))
            return index            
        self.model1.index1 = Set(dimen=2, initialize=index_rule)
                
        self.model1.X1 = Var(self.model1.index1, within=NonNegativeReals)
        
        self.all_data['period'] = self.all_data.index.get_level_values(2)
        
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)
        
    def solve(self):
        opt = SolverFactory('cbc') #Here we use the cplex, other solvers such as cbc solver possible -- open source software
        self.results = opt.solve(self.model1,tee=True) #We solve a problem, but do not show the solver output
        

t1 = optimization_stochastic()

In [ ]:
%%time
class optimization_deterministic:
    def __init__(self):
        c = 0
            self.data1 = pd.read_parquet(path_data + 'barkbeetle_deterministic.parquet', engine='pyarrow')
        self.area = 901.238
        #This is here to keep the dataframes seem similar. Could adjust.
        self.combinations = 1
        self.data1 = self.data1[self.data1['sample']<=self.combinations]
        self.data = self.data1
        
        data1 = self.data.set_index(['stand','schedule','sample','period'])
        stands = list(data1.loc[slice(None),slice(None),0,1].index.get_level_values(0))
        self.data = self.data[self.data['stand'].isin(stands)]

        self.data_opt = self.data

        self.all_data = self.data_opt

        self.Index_values = self.all_data.set_index(['stand','schedule']).index.unique()
        self.Index_sample_year = self.all_data.set_index(['sample','period']).index.unique()
        self.all_data = self.all_data.set_index(['stand','schedule','sample','period'])
        
        self.all_data = self.all_data.fillna(0)
        
        self.createModel()
        self.ADD_OBJECTIVE_FUNCTION()

    def ADD_OBJECTIVE_FUNCTION(self):
        self.model1.NPV= Var(within=NonNegativeReals)
        def NPV_INVENTORY(model1):
            row_sum = sum(sum( (self.all_data.npv.loc[(s,r,sam,12.0)]*self.model1.X1[(s,r)]*self.area)  for (s,r) in self.model1.index1) for sam in range(0,self.combinations))/self.combinations
            return self.model1.NPV ==row_sum
        self.model1.NPV_INV= Constraint(rule=NPV_INVENTORY)

        self.model1.Periodic_volume= Var(self.model1.year,self.model1.sample,within=NonNegativeReals)
        def PERIOD_INVENTORY(model1,p,sam):
            row_sum = sum( (self.all_data.vol_t.loc[(s,r,sam,p)]*self.model1.X1[(s,r)]*self.area)  for (s,r) in self.model1.index1)
            return self.model1.Periodic_volume[p,sam] ==row_sum
        self.model1.Period_INV= Constraint(self.model1.year,self.model1.sample,rule=PERIOD_INVENTORY)

      
        self.model1.DEC_inc = Var(self.model1.sample,within=NonNegativeReals)
        # INC constraints
        def inc_bounded_rule(model1, sam,p):
            INC2 = sum(
                (self.all_data.vol_t.loc[(s,r,sam,p)] ) * self.model1.X1[(s, r)] *self.area
                for (s,r) in self.model1.index1)
            return INC2 >= self.model1.DEC_inc[sam]

        self.model1.INC_bounded = Constraint(self.Index_sample_year, rule=inc_bounded_rule)

        self.model1.DEC_inc_limit = Param(default = 3000,mutable = True)
        # INC constraints
        def inc_bounded_limit_rule(model1, sam,p):
            INC2 = sum(
                (self.all_data.vol_t.loc[(s,r,sam,p)] ) * self.model1.X1[(s, r)] *self.area
                for (s,r) in self.model1.index1)
            return INC2 >= self.model1.DEC_inc_limit
        
        self.model1.INC_bounded_limit = Constraint(self.Index_sample_year, rule=inc_bounded_limit_rule)


        def outcome_rule(model1):
            NPV = self.model1.NPV
            return NPV
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)
                                    
    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        self.model1 = ConcreteModel()
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0))))
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(3))))
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.sample = Set(initialize = list(set(self.all_data.index.get_level_values(2))))
        self.model1.Index_values = self.Index_values

        
        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: #stand_set
                index.append((s,r))
            return index            
        self.model1.index1 = Set(dimen=2, initialize=index_rule)
        
        self.model1.X1 = Var(self.model1.index1, within=NonNegativeReals)
        
        self.all_data['period'] = self.all_data.index.get_level_values(2)
        
        #objective function: Can add if needednew perspective is 
        
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)
        
    def solve(self):
        opt = SolverFactory('cbc') #Here we use the cplex solver, other solvers such as cbc solver possible -- open source software
        self.results = opt.solve(self.model1,tee=True) #We solve a problem, but do not show the solver output

t2 = optimization_deterministic()

In [ ]:
%%time
class optimization_robust:
    def __init__(self):
        c = 0
        self.data1 = pd.read_parquet(path_data + 'barkbeetle_deterministic.parquet', engine='pyarrow')
        self.area = 901.238
        #This is to keep the dataframes seem similar. Could adjust.
        self.combinations = 1
        self.data1 = self.data1[self.data1['sample']<=self.combinations]
        self.data = self.data1
        
        data1 = self.data.set_index(['stand','schedule','sample','period'])
        stands = list(data1.loc[slice(None),slice(None),0,1].index.get_level_values(0))
        self.data = self.data[self.data['stand'].isin(stands)]

        self.data_opt = self.data

        self.all_data = self.data_opt

        self.Index_values = self.all_data.set_index(['stand','schedule']).index.unique()
        self.Index_sample_year = self.all_data.set_index(['sample','period']).index.unique()
        self.all_data = self.all_data.set_index(['stand','schedule','sample','period'])
        
        self.all_data = self.all_data.fillna(0)
        
        self.createModel()
        self.ADD_OBJECTIVE_FUNCTION()

    def ADD_OBJECTIVE_FUNCTION(self):
        self.model1.NPV= Var(within=NonNegativeReals)
        def NPV_INVENTORY(model1):
            row_sum = sum(sum( (self.all_data.npv.loc[(s,r,sam,12.0)]*self.model1.X1[(s,r)]*self.area)  for (s,r) in self.model1.index1) for sam in range(0,self.combinations))/self.combinations
            return self.model1.NPV ==row_sum
        self.model1.NPV_INV= Constraint(rule=NPV_INVENTORY)

        self.model1.Periodic_volume= Var(self.model1.year,self.model1.sample,within=NonNegativeReals)
        def PERIOD_INVENTORY(model1,p,sam):
            row_sum = sum( (self.all_data.vol_t.loc[(s,r,sam,p)]*self.model1.X1[(s,r)]*self.area)  for (s,r) in self.model1.index1)
            return self.model1.Periodic_volume[p,sam] ==row_sum
        self.model1.Period_INV= Constraint(self.model1.year,self.model1.sample,rule=PERIOD_INVENTORY)

        self.model1.Gamma_t = Param(self.model1.year,default = 0.5, mutable =True)
        
        self.model1.u_t = Var(self.model1.year,within= NonNegativeReals)
        self.model1.w_srt = Var(self.model1.index1,self.model1.year,within= NonNegativeReals)
      
        self.model1.DEC_inc = Var(self.model1.sample,within=NonNegativeReals)
        self.model1.DEC_inc_robust = Param(default=3000,mutable = True)
        
        # INC constraints - From deterministic version.
        #def inc_bounded_rule(model1, sam,p):
        #    INC2 = sum(
        #        (self.all_data.vol_t.loc[(s,r,sam,p)] ) * self.model1.X1[(s, r)] 
        #        for (s,r) in self.model1.index1)
        #    return INC2 >= self.model1.DEC_inc[sam]
        #self.model1.INC_bounded = Constraint(self.Index_sample_year, rule=inc_bounded_rule)

        # Gamma INC constraints
        def inc_bounded_robust_rule(model1, sam,p):
            INC2 = sum((self.all_data.vol_t.loc[(s,r,sam,p)] ) * self.model1.X1[(s, r)]*self.area for (s,r) in self.model1.index1) - (
                self.model1.Gamma_t[p]*self.model1.u_t[p]) - (sum(self.model1.w_srt[s,r,p] for (s,r) in self.model1.index1))
            return INC2 >= self.model1.DEC_inc_robust
        self.model1.INC_bounded = Constraint(self.Index_sample_year, rule=inc_bounded_robust_rule)

        #Additional robust constraints #Assuming 2% volume damage from bark beetle. Could be more specific
        def robust_rule_1(model1, sam,p,s,r):
            INC_R = 0.02*((self.all_data.vol_t.loc[(s,r,sam,p)] ) * self.model1.X1[(s, r)]*self.area)
            return self.model1.u_t[p]+self.model1.w_srt[s,r,p] >=INC_R
        self.model1.Robust_rule_one= Constraint(self.Index_sample_year, self.model1.index1, rule=robust_rule_1)

        def outcome_rule(model1):
            NPV = self.model1.NPV
            return NPV
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)
                                    
    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        self.model1 = ConcreteModel()
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0))))
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(3))))
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.sample = Set(initialize = list(set(self.all_data.index.get_level_values(2))))
        self.model1.Index_values = self.Index_values

        
        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: #stand_set
                index.append((s,r))
            return index            
        self.model1.index1 = Set(dimen=2, initialize=index_rule)
        
        self.model1.X1 = Var(self.model1.index1, within=NonNegativeReals)
        
        self.all_data['period'] = self.all_data.index.get_level_values(2)
        
        #objective function: Can add if needednew perspective is 
        
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)
        
    def solve(self):
        opt = SolverFactory('cbc') #Here we use the cplex solver, other solvers such as cbc solver possible -- open source software
        self.results = opt.solve(self.model1,tee=True) #We solve a problem, but do not show the solver output

t3 = optimization_robust()

In [ ]:
%%time
#Maximize only for NPV: 
#Minmimize deviations -- 3 approaches
#Stochastic:
TYPE = "MAX_NPV"
t1.model1.target_down = 0
def outcome_rule(model1):
    NPV = t1.model1.NPV-t1.model1.CVAR_down_Tot *1000
    return NPV
t1.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)
t1.solve()

#Deterministic
t2.model1.DEC_inc_limit = 0
t2.solve()


#Robust
t3.model1.DEC_inc_robust = 0

for p in t3.model1.year:
    t3.model1.Gamma_t[p] = 150# Need to consider how to select the gamma parameter. Could reflect on Palma and nelson.
t3.solve()



In [ ]:
%%time 
#Calculation of core values:
#Resampling to ensure anticipatory planning doesn't occur.

# Optimized function to generate random values for sample
def sample_random(size):
    return np.random.choice(range(100), size=size, replace=False)

# Optimized version of create_sample
def create_sample(df):
    df_reset = df.reset_index()
    unique_stands = df_reset['stand'].unique()
    
    # Loop through each stand and replace the sample values
    for stand in unique_stands:
        # Create a mask for the current stand
        mask = df_reset['stand'] == stand
        
        # Get the unique sample values for this stand
        unique_samples = df_reset.loc[mask, 'sample'].unique()
        
        # Generate random values for the current stand's sample values
        #random_mapping = dict(zip(unique_samples, sample_random(100)))#len(unique_samples))))
        
        df_x = df_reset[df_reset['stand'] ==stand]#, 'sample'] = df_reset4.loc[mask, 'sample'].map(random_mapping)
        sample=sample_random(100)
        df_x =df_x[df_x['sample'].isin(sample)]
        order = {s: i for i, s in enumerate(sample)}

        # sort df_x according to sample order
        df_x = df_x.sort_values(
            by='sample',
            key=lambda s: s.map(order)
        )
        
        df_x['sample'] = pd.factorize(df_x['sample'])[0]
        
        if stand == unique_stands[0]:
            df_y = df_x
        else:
            df_y = pd.concat([df_y,df_x])
    return df_y

# Constructing a sample:
df_reset = create_sample(df=t1.data_orig)

df_reset = df_reset[df_reset['sample']!=-1]
df_reset = df_reset.set_index(['stand', 'schedule','sample', 'period'])

# Pre-fetching all necessary data to avoid repeated lookups
npv_data = df_reset.npv
npv_data_det = t2.all_data.npv

VOL_t_data = df_reset.vol_t
VOL_t_data_det = t2.all_data.vol_t

# Convert X1_values to a Pandas DataFrame
#T1 -- is the stochastic solution
T1_values_dict = {index: t1.model1.X1[index].value for index in t1.model1.X1}
T1_values_df = pd.DataFrame.from_dict(T1_values_dict, orient='index', columns=['T1_value'])
T1_values_df.index = pd.MultiIndex.from_tuples(T1_values_df.index)  # Convert index to MultiIndex

#T2 -- is the deterministic solution
T2_values_dict = {index: t2.model1.X1[index].value for index in t2.model1.X1}
T2_values_df = pd.DataFrame.from_dict(T2_values_dict, orient='index', columns=['T2_value'])
T2_values_df.index = pd.MultiIndex.from_tuples(T2_values_df.index)  # Convert index to MultiIndex if (s, r) is a tuple

#T3 -- is the robust solution
T3_values_dict = {index: t3.model1.X1[index].value for index in t3.model1.X1}
T3_values_df = pd.DataFrame.from_dict(T3_values_dict, orient='index', columns=['T3_value'])
T3_values_df.index = pd.MultiIndex.from_tuples(T3_values_df.index)  # Convert index to MultiIndex if (s, r) is a tuple

T1_values_df = T1_values_df.copy()
T2_values_df = T2_values_df.copy()
T3_values_df = T3_values_df.copy()

T1_values_df.index = T1_values_df.index.set_names(['stand', 'schedule'])
T2_values_df.index = T2_values_df.index.set_names(['stand', 'schedule'])
T3_values_df.index = T3_values_df.index.set_names(['stand', 'schedule'])

npv_sto,npv_det, npv_rob = [],[],[]
ef_sto, ef_det, ef_rob = {},{},{}

for i in range(0, 100):
    npv_sto =npv_sto+[(float(T1_values_df.mul(npv_data.loc[slice(None), slice(None), i, 1], axis=0).sum(axis=0).item()))*901.238]
    npv_det =npv_det+[(float(T2_values_df.mul(npv_data.loc[slice(None), slice(None), i, 1], axis=0).sum(axis=0).item()))*901.238]
    npv_rob =npv_rob+[(float(T3_values_df.mul(npv_data.loc[slice(None), slice(None), i, 1], axis=0).sum(axis=0).item()))*901.238]

for k in range(1,13):
    ef_sto[k] = [float(T1_values_df.mul(VOL_t_data.loc[slice(None), slice(None), i, k], axis=0).sum(axis=0).item())*901.238   for i in range(0, 100)]
    ef_det[k] = [float(T2_values_df.mul(VOL_t_data.loc[slice(None), slice(None), i, k], axis=0).sum(axis=0).item())*901.238   for i in range(0, 100)]
    ef_rob[k] = [float(T3_values_df.mul(VOL_t_data.loc[slice(None), slice(None), i, k], axis=0).sum(axis=0).item())*901.238   for i in range(0, 100)]
    
print("Stochastic",(float(T1_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))
print("Deterministic", (float(T2_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))
print("Robust", (float(T3_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))

print("Perceived decline in stochastic solution:", (float(T1_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item()))/(float(T2_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))

print("Stochastic",(np.mean([npv_sto])))
print("Deterministic", (np.mean([npv_det])))
print("Robust", (np.mean([npv_rob])))
print("Stochastic solution improvement:", (np.mean([npv_sto]))/(np.mean([npv_det])))
print("Stochastic solution improvement:", (np.mean([npv_sto]))/(np.mean([npv_rob])))


In [ ]:
%%time

##NPV ANALYSIS -- constructing the figures that present the distribution of the NPV and the scenario wise comparison

sto_color = "#0072B2"  # blue
rob_color = "#D55E00"  # vermillion
harvest_codes = {18,19,20,21,22,23,24,25,26}

def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

# scenario-wise NPVs
npv_detx = np.array(npv_det)
npv_stox = np.array(npv_sto)
npv_robx = np.array(npv_rob)

diff_sorted = np.sort(npv_stox - npv_detx)
diff_sorted_rob = np.sort(npv_stox - npv_robx)

colors = ['tab:blue' if d > 0 else 'tab:red' for d in diff_sorted]


def lighten(color, amount=0.5):
    c = mcolors.to_rgb(color)
    return tuple(1 - amount*(1 - x) for x in c)

light_blue = lighten('tab:blue', 0.4)
light_red  = lighten('tab:red', 0.4)

colors_rob = [light_blue if d > 0 else light_red for d in diff_sorted_rob]

# scenario-wise NPVs
npv_detx = np.array(npv_det)
npv_stox = np.array(npv_sto)
npv_robx = np.array(npv_rob)

# ---- figure with two panels ----
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

# --- (a) Distribution (hist + optional KDE) ---
ax = axes[0]
# Define one base color per scenario
colorsX = {    "Stochastic": "#1f77b4",   "Deterministic": "#d62728",    "Robust": "#2ca02c","BOTH": "red"}

all_vals = np.concatenate([npv_stox, npv_robx, npv_detx])
bins = np.histogram_bin_edges(all_vals, bins="fd")
bins = 50
# Histograms (light + transparent)
ax.hist(npv_stox, bins=bins, density=True,  color=colorsX["Stochastic"], alpha=0.30, label="Stochastic")
if TYPE != "MAX_NPV":
    ax.hist(npv_detx, bins=bins, density=True, color=colorsX["Deterministic"], alpha=0.30, label="Deterministic")
    ax.hist(npv_robx, bins=bins, density=True, color=colorsX["Robust"], alpha=0.30, label="Robust")
else:
    ax.hist(npv_robx, bins=bins, density=True, color=colorsX["BOTH"], alpha=0.30, label="Deterministic and Robust")
# KDE lines (same color, darker + thicker)
xs = np.linspace(all_vals.min(), all_vals.max(), 300)

ax.plot(xs, gaussian_kde(npv_stox)(xs), color=colorsX["Stochastic"], linewidth=2.5)
if TYPE != "MAX_NPV":
    ax.plot(xs, gaussian_kde(npv_detx)(xs), color=colorsX["Deterministic"], linewidth=2.5)
    ax.plot(xs, gaussian_kde(npv_robx)(xs), color=colorsX["Robust"], linewidth=2.5)
else:
    ax.plot(xs, gaussian_kde(npv_robx)(xs), color=colorsX["BOTH"], linewidth=2.5)
    
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f"{x/1000000:.0f}"))
ax.set_xlabel("Net Present Value (M NOK)")
ax.set_ylabel("Density")
ax.set_title("NPV distribution across scenarios")
ax.legend()
ax.grid(True, alpha=0.3)


# --- (b) Win–loss by scenario (rotated) ---
ax = axes[1]

y = np.arange(len(diff_sorted))
ax.scatter(diff_sorted, y, c=colors, label="Stochastic - Deterministic")

if TYPE != "MAX_NPV":
    ax.scatter(diff_sorted_rob, y, c=colors_rob, 
               label="Robust - Deterministic")
ax.axvline(0, linewidth=1)

ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f"{x/1000000:.0f}"))
ax.set_ylabel("Scenario (sorted)")
ax.set_xlabel("NPV difference (M NOK)")
ax.set_title("Scenario-wise win–loss")

ax.grid(True, alpha=0.3)

if TYPE != "MAX_NPV":
    ax.legend(frameon=False, loc="upper left")

# panel labels
axes[0].text(    -0.12, 1.05, "(a)",    transform=axes[0].transAxes,    fontsize=12,    va="top")
axes[1].text(    -0.12, 1.05, "(b)",    transform=axes[1].transAxes,    fontsize=12,    va="top")

fig.savefig(    path_out+"npv_ecdf_winloss_NPV"+TYPE+".png",    bbox_inches="tight")

plt.show()

def merge_decision(df_reset, values_df, decision_col, label):
    """
    Returns a merged dataframe with one decision column added.
    """

    # decision dataframe
    v = values_df.copy().reset_index()
    v = v.rename(columns={"level_0": "stand", "level_1": "schedule"})
    v = v[["stand", "schedule", decision_col]]
    v = v.rename(columns={decision_col: f"{label}_value"})

    # base dataframe
    base = df_reset.copy().reset_index()

    merged = base.merge(
        v,
        on=["stand", "schedule"],
        how="outer"
    )

    merged = merged.set_index(["stand", "schedule", "sample", "period"])

    return merged

merged_datasets = {}

for label, values_df, decision_col in [
    ("T1", T1_values_df, "T1_value"),
    ("T2", T2_values_df, "T2_value"),
    ("T3", T3_values_df, "T3_value"),
]:
    merged_datasets[label] = merge_decision(
        df_reset,
        values_df,
        decision_col,
        label
    )

# Access them
merged_T1 = merged_datasets["T1"]
merged_T2 = merged_datasets["T2"]
merged_T3 = merged_datasets["T3"]

merged_all = df_reset.copy().reset_index()

for label, values_df, decision_col in [
    ("T1", T1_values_df, "T1_value"),
    ("T2", T2_values_df, "T2_value"),
    ("T3", T3_values_df, "T3_value"),
]:
    v = values_df.copy().reset_index()
    v = v.rename(columns={"level_0": "stand", "level_1": "schedule"})
    v = v[["stand", "schedule", decision_col]]
    v = v.rename(columns={decision_col: f"{label}_value"})

    merged_all = merged_all.merge(v, on=["stand", "schedule"], how="outer")

merged_all = merged_all.set_index(["stand", "schedule", "sample", "period"])

tol = 1e-9  # adjust if needed

m = merged_all.copy()

# difference + boolean flag
m["diff_T1_T2"] = m["T1_value"] - m["T2_value"]
m["is_diff_T1_T2"] = m["diff_T1_T2"].abs() > tol

m[(m["is_diff_T1_T2"]) & ((m["redID"]>0) |(m["trID"]>0) )].loc[(slice(None), slice(None), 0, slice(None)), :]

### t1 -- stochastic
#t2 -- deterministic
#t3 -- Robust

m["diff_T1_T2"] = m["T1_value"] - m["T2_value"]
m["diff_T3_T2"] = m["T3_value"] - m["T2_value"]
m["is_diff_T1_T2"] = m["diff_T1_T2"].abs() > tol
m["is_diff_T3_T2"] = m["diff_T3_T2"].abs() > tol

m[((m["redID"]>0) |(m["trID"]>0) )].loc[(slice(None), slice(None), 0, slice(None)), :]

df = m.reset_index()
df = df[df["sample"] == 0]
df = df[df["trID"].isin(harvest_codes)]

df[~((df["T1_value"] == 0) &
          (df["T2_value"] == 0) &
          (df["T3_value"] == 0))]

print(df[df['period']==1][(df['diff_T3_T2']>0)])
print(df[df['period']==1][(df['diff_T3_T2']<0)])

In [ ]:

differe = "T1_T2"
MAX_JUMP =3
#differe = "T3_T2" # This will not work if running the max NPV case -- as T3 and T2 are the same.

def add_flow_columns(
    df1,
    period_col="period",
    diff_col="diff_"+differe,
    group_col="stand",
    schedule_col="schedule",
    max_jump=MAX_JUMP,
):
    out = df1.copy()

    # output columns (summary on one "anchor" row per stand)
    out["time"] = np.nan
    out["difference"] = np.nan
    out["from to"] = np.nan
    out["from to"] = out["from to"].astype("object")

    # diagnostics (flags can be on multiple rows)
    out["flag_jump_gt3"] = False
    out["jump_abs"] = np.nan
    out["flag_schedule_period_span_gt3"] = False
    out["schedule_period_span"] = np.nan

    # ---- 1) stand-level flow summary + jump flag ----
    for stand, g in out.groupby(group_col, sort=False):
        neg = g[g[diff_col] < 0]
        pos = g[g[diff_col] > 0]

        if not neg.empty:
            idx_source = neg[diff_col].idxmin()
            source_period = int(out.loc[idx_source, period_col])
        else:
            idx_source = None
            source_period = None

        if not pos.empty:
            idx_sink = pos[diff_col].idxmax()
            sink_period = int(out.loc[idx_sink, period_col])
        else:
            idx_sink = None
            sink_period = None

        if source_period is None and sink_period is None:
            continue

        # fallbacks (same as your current logic)
        if source_period is None:
            source_period = int(sink_period)
            sink_period = max(source_period - 1, 1)

        if sink_period is None:
            sink_period = max(source_period - 1, 1)

        jump_abs = abs(int(sink_period) - int(source_period))
        flag_jump = jump_abs > max_jump

        # anchor row choice (same as your current logic)
        anchor_period = max(source_period, sink_period)
        if anchor_period == sink_period and idx_sink is not None:
            idx_anchor = idx_sink
        elif idx_source is not None:
            idx_anchor = idx_source
        else:
            idx_anchor = pos[diff_col].idxmax()

        out.loc[idx_anchor, "time"] = int(source_period != sink_period)
        out.loc[idx_anchor, "difference"] = out.loc[idx_anchor, diff_col]
        out.loc[idx_anchor, "from to"] = f"{source_period} to {sink_period}"

        out.loc[idx_anchor, "jump_abs"] = jump_abs
        out.loc[idx_anchor, "flag_jump_gt3"] = flag_jump

    # ---- 2) stand+schedule duplicate check: periods far apart ----
    # mark ALL rows belonging to a (stand, schedule) group where span > max_jump AND group has >1 row
    if schedule_col in out.columns:
        sched_stats = (
            out.groupby([group_col, schedule_col], sort=False)[period_col]
               .agg(n="size", pmin="min", pmax="max")
               .reset_index()
        )
        sched_stats["period_span"] = (sched_stats["pmax"] - sched_stats["pmin"]).astype(int)
        sched_stats["flag_span_gt3"] = (sched_stats["n"] > 1) & (sched_stats["period_span"] > max_jump)

        flagged = sched_stats.loc[sched_stats["flag_span_gt3"], [group_col, schedule_col, "period_span"]]

        if not flagged.empty:
            out = out.merge(flagged, on=[group_col, schedule_col], how="left")
            out["schedule_period_span"] = out["period_span"]
            out["flag_schedule_period_span_gt3"] = out["period_span"].notna()
            out.drop(columns=["period_span"], inplace=True)
        else:
            out["schedule_period_span"] = np.nan
            out["flag_schedule_period_span_gt3"] = False

    return out

df1 = (
    df[df["diff_"+differe] != 0]
      .sort_values(["stand",  "schedule","period"])
      .reset_index(drop=True)
)
#df1=df1[df1['period']>6]
MAX_JUMP = 3  # anything greater than this should be checked

df2 = add_flow_columns(df1, period_col="period", diff_col="diff_"+differe, group_col="stand")

flows = df2.dropna(subset=["from to"]).copy()

tmp = flows["from to"].str.extract(r"^\s*(\d+)\s+to\s+(\d+)\s*$")
flows["from_period"] = tmp[0].astype(int)
flows["to_period"] = tmp[1].astype(int)

# ignore same-period flows
flows = flows[flows["from_period"] != flows["to_period"]]

# use absolute value
flows["abs_difference"] = flows["difference"].abs()

flow_sum = (
    flows.groupby(["from_period", "to_period"], as_index=False)["abs_difference"]
         .sum()
)

# keep NaN for non-existing transitions
M = (
    flow_sum.pivot(index="from_period", columns="to_period", values="abs_difference")
            .reindex(index=range(1, 13), columns=range(1, 13))
)

df1 = (
    df[df["diff_"+differe] != 0]
      .sort_values(["stand",  "schedule","period"])
      .reset_index(drop=True)
)
df1=df1[df1['period']>6]
MAX_JUMP = 3  # anything greater than this should be checked

df2 = add_flow_columns(df1, period_col="period", diff_col="diff_"+differe, group_col="stand")

flows = df2.dropna(subset=["from to"]).copy()

tmp = flows["from to"].str.extract(r"^\s*(\d+)\s+to\s+(\d+)\s*$")
flows["from_period"] = tmp[0].astype(int)
flows["to_period"] = tmp[1].astype(int)

# ignore same-period flows
flows = flows[flows["from_period"] != flows["to_period"]]

# use absolute value
flows["abs_difference"] = flows["difference"].abs()

flow_sum = (
    flows.groupby(["from_period", "to_period"], as_index=False)["abs_difference"]
         .sum()
)

# keep NaN for non-existing transitions
Ma = (
    flow_sum.pivot(index="from_period", columns="to_period", values="abs_difference")
            .reindex(index=range(1, 13), columns=range(1, 13))
)

rows = [11, 12]
cols = [11, 12]

M.loc[rows, cols] = Ma.loc[rows, cols]


fig, axes = plt.subplots(
    1, 1,
    figsize=(6, 6),   # width roughly 2x height
    sharex=True,
    sharey=True
)

vmax = np.nanmax([np.nanmax(M.values*902),
                  np.nanmax(M.values*902)])

# ---------- First matrix ----------
data = M.values * 902
im0 = axes.imshow(data, aspect="equal", vmin=0, vmax=vmax)

axes.set_xticks(range(12))
axes.set_xticklabels(range(1, 13))
axes.set_yticks(range(12))
axes.set_yticklabels(range(1, 13))
axes.set_xlabel("To period")
axes.set_ylabel("From period")

axes.set_xlim(-0.5, 11.5)
axes.set_ylim(11.5, -0.5)

norm = im0.norm
cmap = im0.cmap

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        value = data[i, j]
        if not np.isnan(value) and value > 0:
            rgba = cmap(norm(value))
            brightness = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
            text_color = "black" if brightness > 0.5 else "white"

            axes.text(
                j, i,
                f"{value:,.0f}",
                ha="center",
                va="center",
                color=text_color,
                fontsize=8,
                fontweight="bold"
            )
            n = data.shape[0]  # assuming square matrix            
            axes.plot(
                [-0.5, n - 0.5],
                [-0.5, n - 0.5],
                color="black",
                linewidth=2,
                linestyle="--"   # optional
            )   

# ---------- Shared colorbar ----------
cbar = fig.colorbar(
    im0,
    ax=axes,
    fraction=0.045,   # keeps colorbar proportional
    pad=0.04,
    label="Area with a change of harvesting (ha)"
)


plt.savefig(
    path_out + "BALANCE" + TYPE + "_" + differe + ".png",
    dpi=300
)
plt.show()


# Conduct even flow analysis

In [ ]:
%%time
#Maximize for 3000 m3
#Minmimize deviations -- 3 approaches
#Stochastic:
TYPE = "EVEN_FLOW"
t1.model1.target_down = 600000*5#3000#50000/900
def outcome_rule(model1):
    NPV = t1.model1.NPV-t1.model1.CVAR_down_Tot *1000
    return NPV
t1.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)
t1.solve()

#Deterministic
t2.model1.DEC_inc_limit = 600000*5#3000#50000/900
t2.solve()


#Robust
t3.model1.DEC_inc_robust = 600000*5#3000#50000/900

for p in t3.model1.year:
    t3.model1.Gamma_t[p] = 3.5# Need to consider how to select the gamma parameter. Could reflect on Palma and nelson.
t3.solve()


In [ ]:
%%time 
#Calculation of core values:
#Resampling to ensure anticipatory planning doesn't occur.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Optimized function to generate random values for sample
def sample_random(size):
    return np.random.choice(range(100), size=size, replace=False)

# Optimized version of create_sample
def create_sample(df):
    df_reset = df.reset_index()
    unique_stands = df_reset['stand'].unique()
    
    # Loop through each stand and replace the sample values
    for stand in unique_stands:
        # Create a mask for the current stand
        mask = df_reset['stand'] == stand
        
        # Get the unique sample values for this stand
        unique_samples = df_reset.loc[mask, 'sample'].unique()
        
        # Generate random values for the current stand's sample values
        #random_mapping = dict(zip(unique_samples, sample_random(100)))#len(unique_samples))))
        
        df_x = df_reset[df_reset['stand'] ==stand]#, 'sample'] = df_reset4.loc[mask, 'sample'].map(random_mapping)
        sample=sample_random(100)
        df_x =df_x[df_x['sample'].isin(sample)]
        order = {s: i for i, s in enumerate(sample)}

        # sort df_x according to sample order
        df_x = df_x.sort_values(
            by='sample',
            key=lambda s: s.map(order)
        )
        
        df_x['sample'] = pd.factorize(df_x['sample'])[0]
        
        if stand == unique_stands[0]:
            df_y = df_x
        else:
            df_y = pd.concat([df_y,df_x])
    return df_y

# Constructing a sample:
df_reset = create_sample(df=t1.data_orig)

df_reset = df_reset[df_reset['sample']!=-1]
df_reset = df_reset.set_index(['stand', 'schedule','sample', 'period'])

# Pre-fetching all necessary data to avoid repeated lookups
npv_data = df_reset.npv
npv_data_det = t2.all_data.npv

VOL_t_data = df_reset.vol_t
VOL_t_data_det = t2.all_data.vol_t

# Convert X1_values to a Pandas DataFrame
#T1 -- is the stochastic solution
T1_values_dict = {index: t1.model1.X1[index].value for index in t1.model1.X1}
T1_values_df = pd.DataFrame.from_dict(T1_values_dict, orient='index', columns=['T1_value'])
T1_values_df.index = pd.MultiIndex.from_tuples(T1_values_df.index)  # Convert index to MultiIndex

#T2 -- is the deterministic solution
T2_values_dict = {index: t2.model1.X1[index].value for index in t2.model1.X1}
T2_values_df = pd.DataFrame.from_dict(T2_values_dict, orient='index', columns=['T2_value'])
T2_values_df.index = pd.MultiIndex.from_tuples(T2_values_df.index)  # Convert index to MultiIndex if (s, r) is a tuple

#T3 -- is the robust solution
T3_values_dict = {index: t3.model1.X1[index].value for index in t3.model1.X1}
T3_values_df = pd.DataFrame.from_dict(T3_values_dict, orient='index', columns=['T3_value'])
T3_values_df.index = pd.MultiIndex.from_tuples(T3_values_df.index)  # Convert index to MultiIndex if (s, r) is a tuple

T1_values_df = T1_values_df.copy()
T2_values_df = T2_values_df.copy()
T3_values_df = T3_values_df.copy()

T1_values_df.index = T1_values_df.index.set_names(['stand', 'schedule'])
T2_values_df.index = T2_values_df.index.set_names(['stand', 'schedule'])
T3_values_df.index = T3_values_df.index.set_names(['stand', 'schedule'])

npv_sto,npv_det, npv_rob = [],[],[]
ef_sto, ef_det, ef_rob = {},{},{}

for i in range(0, 100):
    npv_sto =npv_sto+[(float(T1_values_df.mul(npv_data.loc[slice(None), slice(None), i, 1], axis=0).sum(axis=0).item()))*901.238]
    npv_det =npv_det+[(float(T2_values_df.mul(npv_data.loc[slice(None), slice(None), i, 1], axis=0).sum(axis=0).item()))*901.238]
    npv_rob =npv_rob+[(float(T3_values_df.mul(npv_data.loc[slice(None), slice(None), i, 1], axis=0).sum(axis=0).item()))*901.238]

for k in range(1,13):
    ef_sto[k] = [float(T1_values_df.mul(VOL_t_data.loc[slice(None), slice(None), i, k], axis=0).sum(axis=0).item())*901.238   for i in range(0, 100)]
    ef_det[k] = [float(T2_values_df.mul(VOL_t_data.loc[slice(None), slice(None), i, k], axis=0).sum(axis=0).item())*901.238   for i in range(0, 100)]
    ef_rob[k] = [float(T3_values_df.mul(VOL_t_data.loc[slice(None), slice(None), i, k], axis=0).sum(axis=0).item())*901.238   for i in range(0, 100)]
    
print("Stochastic",(float(T1_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))
print("Deterministic", (float(T2_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))
print("Robust", (float(T3_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))

print("Perceived decline in stochastic solution:", (float(T1_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item()))/(float(T2_values_df.mul(npv_data_det.loc[slice(None), slice(None), 0, 1], axis=0).sum(axis=0).item())))

print("Stochastic",(np.mean([npv_sto])))
print("Deterministic", (np.mean([npv_det])))
print("Robust", (np.mean([npv_rob])))
print("Stochastic solution improvement:", (np.mean([npv_sto]))/(np.mean([npv_det])))
print("Stochastic solution improvement:", (np.mean([npv_sto]))/(np.mean([npv_rob])))


In [ ]:
%%time

##NPV ANALYSIS -- constructing the figures that present the distribution of the NPV and the scenario wise comparison
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from matplotlib.ticker import MaxNLocator
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sto_color = "#0072B2"  # blue
rob_color = "#D55E00"  # vermillion

def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

# scenario-wise NPVs
npv_detx = np.array(npv_det)
npv_stox = np.array(npv_sto)
npv_robx = np.array(npv_rob)

diff_sorted = np.sort(npv_stox - npv_detx)
diff_sorted_rob = np.sort(npv_stox - npv_robx)

colors = ['tab:blue' if d > 0 else 'tab:red' for d in diff_sorted]


def lighten(color, amount=0.5):
    c = mcolors.to_rgb(color)
    return tuple(1 - amount*(1 - x) for x in c)

light_blue = lighten('tab:blue', 0.4)
light_red  = lighten('tab:red', 0.4)

colors_rob = [light_blue if d > 0 else light_red for d in diff_sorted_rob]

# scenario-wise NPVs
npv_detx = np.array(npv_det)
npv_stox = np.array(npv_sto)
npv_robx = np.array(npv_rob)

# ---- figure with two panels ----
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

# --- (a) Distribution (hist + optional KDE) ---
ax = axes[0]
# Define one base color per scenario
colorsX = {    "Stochastic": "#1f77b4",   "Deterministic": "#d62728",    "Robust": "#2ca02c","BOTH": "red"}

all_vals = np.concatenate([npv_stox, npv_robx, npv_detx])
bins = np.histogram_bin_edges(all_vals, bins="fd")
bins = 50
# Histograms (light + transparent)
ax.hist(npv_stox, bins=bins, density=True,  color=colorsX["Stochastic"], alpha=0.30, label="Stochastic")
if TYPE != "MAX_NPV":
    ax.hist(npv_detx, bins=bins, density=True, color=colorsX["Deterministic"], alpha=0.30, label="Deterministic")
    ax.hist(npv_robx, bins=bins, density=True, color=colorsX["Robust"], alpha=0.30, label="Robust")
else:
    ax.hist(npv_robx, bins=bins, density=True, color=colorsX["BOTH"], alpha=0.30, label="Deterministic and Robust")
# KDE lines (same color, darker + thicker)
xs = np.linspace(all_vals.min(), all_vals.max(), 300)

ax.plot(xs, gaussian_kde(npv_stox)(xs), color=colorsX["Stochastic"], linewidth=2.5)
if TYPE != "MAX_NPV":
    ax.plot(xs, gaussian_kde(npv_detx)(xs), color=colorsX["Deterministic"], linewidth=2.5)
    ax.plot(xs, gaussian_kde(npv_robx)(xs), color=colorsX["Robust"], linewidth=2.5)
else:
    ax.plot(xs, gaussian_kde(npv_robx)(xs), color=colorsX["BOTH"], linewidth=2.5)
    
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f"{x/1000000:.0f}"))
ax.set_xlabel("Net Present Value (M NOK)")
ax.set_ylabel("Density")
ax.set_title("NPV distribution across scenarios")
ax.legend()
ax.grid(True, alpha=0.3)


# --- (b) Win–loss by scenario (rotated) ---
ax = axes[1]

y = np.arange(len(diff_sorted))
ax.scatter(diff_sorted, y, c=colors, label="Stochastic - Deterministic")

if TYPE != "MAX_NPV":
    ax.scatter(diff_sorted_rob, y, c=colors_rob, 
               label="Robust - Deterministic")
ax.axvline(0, linewidth=1)

ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f"{x/1000000:.0f}"))
ax.set_ylabel("Scenario (sorted)")
ax.set_xlabel("NPV difference (M NOK)")
ax.set_title("Scenario-wise win–loss")

ax.grid(True, alpha=0.3)

if TYPE != "MAX_NPV":
    ax.legend(frameon=False, loc="upper left")

# panel labels
axes[0].text(    -0.12, 1.05, "(a)",    transform=axes[0].transAxes,    fontsize=12,    va="top")
axes[1].text(    -0.12, 1.05, "(b)",    transform=axes[1].transAxes,    fontsize=12,    va="top")

fig.savefig(    path_out+"npv_ecdf_winloss_NPV"+TYPE+".png",    bbox_inches="tight")

plt.show()


def merge_decision(df_reset, values_df, decision_col, label):
    """
    Returns a merged dataframe with one decision column added.
    """

    # decision dataframe
    v = values_df.copy().reset_index()
    v = v.rename(columns={"level_0": "stand", "level_1": "schedule"})
    v = v[["stand", "schedule", decision_col]]
    v = v.rename(columns={decision_col: f"{label}_value"})

    # base dataframe
    base = df_reset.copy().reset_index()

    merged = base.merge(
        v,
        on=["stand", "schedule"],
        how="outer"
    )

    merged = merged.set_index(["stand", "schedule", "sample", "period"])

    return merged

merged_datasets = {}

for label, values_df, decision_col in [
    ("T1", T1_values_df, "T1_value"),
    ("T2", T2_values_df, "T2_value"),
    ("T3", T3_values_df, "T3_value"),
]:
    merged_datasets[label] = merge_decision(
        df_reset,
        values_df,
        decision_col,
        label
    )

# Access them
merged_T1 = merged_datasets["T1"]
merged_T2 = merged_datasets["T2"]
merged_T3 = merged_datasets["T3"]

merged_all = df_reset.copy().reset_index()

for label, values_df, decision_col in [
    ("T1", T1_values_df, "T1_value"),
    ("T2", T2_values_df, "T2_value"),
    ("T3", T3_values_df, "T3_value"),
]:
    v = values_df.copy().reset_index()
    v = v.rename(columns={"level_0": "stand", "level_1": "schedule"})
    v = v[["stand", "schedule", decision_col]]
    v = v.rename(columns={decision_col: f"{label}_value"})

    merged_all = merged_all.merge(v, on=["stand", "schedule"], how="outer")

merged_all = merged_all.set_index(["stand", "schedule", "sample", "period"])

tol = 1e-9  # adjust if needed

m = merged_all.copy()

# difference + boolean flag
m["diff_T1_T2"] = m["T1_value"] - m["T2_value"]
m["is_diff_T1_T2"] = m["diff_T1_T2"].abs() > tol

m[(m["is_diff_T1_T2"]) & ((m["redID"]>0) |(m["trID"]>0) )].loc[(slice(None), slice(None), 0, slice(None)), :]

### t1 -- stochastic
#t2 -- deterministic
#t3 -- Robust

m["diff_T1_T2"] = m["T1_value"] - m["T2_value"]
m["diff_T3_T2"] = m["T3_value"] - m["T2_value"]
m["is_diff_T1_T2"] = m["diff_T1_T2"].abs() > tol
m["is_diff_T3_T2"] = m["diff_T3_T2"].abs() > tol

m[((m["redID"]>0) |(m["trID"]>0) )].loc[(slice(None), slice(None), 0, slice(None)), :]

df = m.reset_index()
df = df[df["sample"] == 0]
df = df[df["trID"].isin(harvest_codes)]

df[~((df["T1_value"] == 0) &
          (df["T2_value"] == 0) &
          (df["T3_value"] == 0))]

print(df[df['period']==1][(df['diff_T3_T2']>0)])
print(df[df['period']==1][(df['diff_T3_T2']<0)])

# Used to construct the temporal figures, difference from deterministic case. A minor issue remains -- 
# if multiple harvest times, the later harvest times may be removed. A simple approach to extract the values
# is to run without the first few periods, and compare the plots.



In [ ]:
import numpy as np
import pandas as pd
differe = "T1_T2"
#differe = "T3_T2" # This will not work if running the max NPV case -- as T3 and T2 are the same.

def add_flow_columns(
    df1,
    period_col="period",
    diff_col="diff_"+differe,
    group_col="stand",
    schedule_col="schedule",
    max_jump=MAX_JUMP,
):
    out = df1.copy()

    # output columns (summary on one "anchor" row per stand)
    out["time"] = np.nan
    out["difference"] = np.nan
    out["from to"] = np.nan
    out["from to"] = out["from to"].astype("object")

    # diagnostics (flags can be on multiple rows)
    out["flag_jump_gt3"] = False
    out["jump_abs"] = np.nan
    out["flag_schedule_period_span_gt3"] = False
    out["schedule_period_span"] = np.nan

    # ---- 1) stand-level flow summary + jump flag ----
    for stand, g in out.groupby(group_col, sort=False):
        neg = g[g[diff_col] < 0]
        pos = g[g[diff_col] > 0]

        if not neg.empty:
            idx_source = neg[diff_col].idxmin()
            source_period = int(out.loc[idx_source, period_col])
        else:
            idx_source = None
            source_period = None

        if not pos.empty:
            idx_sink = pos[diff_col].idxmax()
            sink_period = int(out.loc[idx_sink, period_col])
        else:
            idx_sink = None
            sink_period = None

        if source_period is None and sink_period is None:
            continue

        # fallbacks (same as your current logic)
        if source_period is None:
            source_period = int(sink_period)
            sink_period = max(source_period - 1, 1)

        if sink_period is None:
            sink_period = max(source_period - 1, 1)

        jump_abs = abs(int(sink_period) - int(source_period))
        flag_jump = jump_abs > max_jump

        # anchor row choice (same as your current logic)
        anchor_period = max(source_period, sink_period)
        if anchor_period == sink_period and idx_sink is not None:
            idx_anchor = idx_sink
        elif idx_source is not None:
            idx_anchor = idx_source
        else:
            idx_anchor = pos[diff_col].idxmax()

        out.loc[idx_anchor, "time"] = int(source_period != sink_period)
        out.loc[idx_anchor, "difference"] = out.loc[idx_anchor, diff_col]
        out.loc[idx_anchor, "from to"] = f"{source_period} to {sink_period}"

        out.loc[idx_anchor, "jump_abs"] = jump_abs
        out.loc[idx_anchor, "flag_jump_gt3"] = flag_jump

    # ---- 2) stand+schedule duplicate check: periods far apart ----
    # mark ALL rows belonging to a (stand, schedule) group where span > max_jump AND group has >1 row
    if schedule_col in out.columns:
        sched_stats = (
            out.groupby([group_col, schedule_col], sort=False)[period_col]
               .agg(n="size", pmin="min", pmax="max")
               .reset_index()
        )
        sched_stats["period_span"] = (sched_stats["pmax"] - sched_stats["pmin"]).astype(int)
        sched_stats["flag_span_gt3"] = (sched_stats["n"] > 1) & (sched_stats["period_span"] > max_jump)

        flagged = sched_stats.loc[sched_stats["flag_span_gt3"], [group_col, schedule_col, "period_span"]]

        if not flagged.empty:
            out = out.merge(flagged, on=[group_col, schedule_col], how="left")
            out["schedule_period_span"] = out["period_span"]
            out["flag_schedule_period_span_gt3"] = out["period_span"].notna()
            out.drop(columns=["period_span"], inplace=True)
        else:
            out["schedule_period_span"] = np.nan
            out["flag_schedule_period_span_gt3"] = False

    return out

df1 = (
    df[df["diff_"+differe] != 0]
      .sort_values(["stand",  "schedule","period"])
      .reset_index(drop=True)
)
#df1=df1[df1['period']>6]
MAX_JUMP = 3  # anything greater than this should be checked

df2 = add_flow_columns(df1, period_col="period", diff_col="diff_"+differe, group_col="stand")

flows = df2.dropna(subset=["from to"]).copy()

tmp = flows["from to"].str.extract(r"^\s*(\d+)\s+to\s+(\d+)\s*$")
flows["from_period"] = tmp[0].astype(int)
flows["to_period"] = tmp[1].astype(int)

# ignore same-period flows
flows = flows[flows["from_period"] != flows["to_period"]]

# use absolute value
flows["abs_difference"] = flows["difference"].abs()

flow_sum = (
    flows.groupby(["from_period", "to_period"], as_index=False)["abs_difference"]
         .sum()
)

# keep NaN for non-existing transitions
M = (
    flow_sum.pivot(index="from_period", columns="to_period", values="abs_difference")
            .reindex(index=range(1, 13), columns=range(1, 13))
)

df1 = (
    df[df["diff_"+differe] != 0]
      .sort_values(["stand",  "schedule","period"])
      .reset_index(drop=True)
)
df1=df1[df1['period']>6]
MAX_JUMP = 3  # anything greater than this should be checked

df2 = add_flow_columns(df1, period_col="period", diff_col="diff_"+differe, group_col="stand")

flows = df2.dropna(subset=["from to"]).copy()

tmp = flows["from to"].str.extract(r"^\s*(\d+)\s+to\s+(\d+)\s*$")
flows["from_period"] = tmp[0].astype(int)
flows["to_period"] = tmp[1].astype(int)

# ignore same-period flows
flows = flows[flows["from_period"] != flows["to_period"]]

# use absolute value
flows["abs_difference"] = flows["difference"].abs()

flow_sum = (
    flows.groupby(["from_period", "to_period"], as_index=False)["abs_difference"]
         .sum()
)

# keep NaN for non-existing transitions
Ma = (
    flow_sum.pivot(index="from_period", columns="to_period", values="abs_difference")
            .reindex(index=range(1, 13), columns=range(1, 13))
)

rows = [11, 12]
cols = [11, 12]

M.loc[rows, cols] = Ma.loc[rows, cols]




differe = "T3_T2"

df1 = (
    df[df["diff_"+differe] != 0]
      .sort_values(["stand",  "schedule","period"])
      .reset_index(drop=True)
)
#df1=df1[df1['period']>6]
MAX_JUMP = 3  # anything greater than this should be checked

df2 = add_flow_columns(df1, period_col="period", diff_col="diff_"+differe, group_col="stand")

flows = df2.dropna(subset=["from to"]).copy()

tmp = flows["from to"].str.extract(r"^\s*(\d+)\s+to\s+(\d+)\s*$")
flows["from_period"] = tmp[0].astype(int)
flows["to_period"] = tmp[1].astype(int)

# ignore same-period flows
flows = flows[flows["from_period"] != flows["to_period"]]

# use absolute value
flows["abs_difference"] = flows["difference"].abs()

flow_sum = (
    flows.groupby(["from_period", "to_period"], as_index=False)["abs_difference"]
         .sum()
)

# keep NaN for non-existing transitions
M1 = (
    flow_sum.pivot(index="from_period", columns="to_period", values="abs_difference")
            .reindex(index=range(1, 13), columns=range(1, 13))
)

df1 = (
    df[df["diff_"+differe] != 0]
      .sort_values(["stand",  "schedule","period"])
      .reset_index(drop=True)
)
df1=df1[df1['period']>6]
MAX_JUMP = 3  # anything greater than this should be checked

df2 = add_flow_columns(df1, period_col="period", diff_col="diff_"+differe, group_col="stand")

flows = df2.dropna(subset=["from to"]).copy()

tmp = flows["from to"].str.extract(r"^\s*(\d+)\s+to\s+(\d+)\s*$")
flows["from_period"] = tmp[0].astype(int)
flows["to_period"] = tmp[1].astype(int)

# ignore same-period flows
flows = flows[flows["from_period"] != flows["to_period"]]

# use absolute value
flows["abs_difference"] = flows["difference"].abs()

flow_sum = (
    flows.groupby(["from_period", "to_period"], as_index=False)["abs_difference"]
         .sum()
)

# keep NaN for non-existing transitions
M1a = (
    flow_sum.pivot(index="from_period", columns="to_period", values="abs_difference")
            .reindex(index=range(1, 13), columns=range(1, 13))
)

rows = [11, 12]
cols = [11, 12]

M1.loc[rows, cols] = M1a.loc[rows, cols]



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axes = plt.subplots(
    1, 2,
    figsize=(12, 6),   # width roughly 2x height
    sharex=True,
    sharey=True
)

vmax = np.nanmax([np.nanmax(M.values*902),
                  np.nanmax(M1.values*902)])

# ---------- First matrix ----------
data = M.values * 902
im0 = axes[0].imshow(data, aspect="equal", vmin=0, vmax=vmax)

axes[0].set_xticks(range(12))
axes[0].set_xticklabels(range(1, 13))
axes[0].set_yticks(range(12))
axes[0].set_yticklabels(range(1, 13))
axes[0].set_xlabel("To period")
axes[0].set_ylabel("From period")
axes[0].set_title("Stochastic v Deterministic")

axes[0].set_xlim(-0.5, 11.5)
axes[0].set_ylim(11.5, -0.5)

norm = im0.norm
cmap = im0.cmap

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        value = data[i, j]
        if not np.isnan(value) and value > 0:
            rgba = cmap(norm(value))
            brightness = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
            text_color = "black" if brightness > 0.5 else "white"

            axes[0].text(
                j, i,
                f"{value:,.0f}",
                ha="center",
                va="center",
                color=text_color,
                fontsize=8,
                fontweight="bold"
            )
            n = data.shape[0]  # assuming square matrix            
            axes[0].plot(
                [-0.5, n - 0.5],
                [-0.5, n - 0.5],
                color="black",
                linewidth=2,
                linestyle="--"   # optional
            )   

# ---------- Second matrix ----------
data1 = M1.values * 902
im1 = axes[1].imshow(data1, aspect="equal", vmin=0, vmax=vmax)

axes[1].set_xticks(range(12))
axes[1].set_xticklabels(range(1, 13))
axes[1].set_yticks(range(12))
axes[1].set_yticklabels(range(1, 13))
axes[1].set_xlabel("To period")
axes[1].set_title("Robust v Deterministic")

axes[1].set_xlim(-0.5, 11.5)
axes[1].set_ylim(11.5, -0.5)

norm = im1.norm
cmap = im1.cmap

for i in range(data1.shape[0]):
    for j in range(data1.shape[1]):
        value = data1[i, j]
        if not np.isnan(value) and value > 0:
            rgba = cmap(norm(value))
            brightness = 0.299*rgba[0] + 0.587*rgba[1] + 0.114*rgba[2]
            text_color = "black" if brightness > 0.5 else "white"

            axes[1].text(
                j, i,
                f"{value:,.0f}",
                ha="center",
                va="center",
                color=text_color,
                fontsize=8,
                fontweight="bold"
            )
            n = data.shape[0]  # assuming square matrix            
            axes[1].plot(
                [-0.5, n - 0.5],
                [-0.5, n - 0.5],
                color="black",
                linewidth=2,
                linestyle="--"   # optional
            )   

# ---------- Shared colorbar ----------
cbar = fig.colorbar(
    im0,
    ax=axes,
    fraction=0.021,   # keeps colorbar proportional
    pad=0.04,
    label="Area with a change of harvesting (ha)"
)


plt.savefig(
    path_out + "BALANCE" + TYPE + "_" + differe + ".png",
    dpi=300
)
plt.show()


In [ ]:
#Not needed for MAX NPV:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#Constructing a version of Figure 6 -- only used for the case when periodic income is important
target = 600000*5
periods = range(1, 13)

per_ha = target
# compute expected shortfall
shortfall = {
    "Deterministic": [],
    "Robust": [],
    "Stochastic": [],
}

for l in periods:
    shortfall["Deterministic"].append(
        100 * np.mean(np.maximum(0, target - np.array(ef_det[l]))/per_ha)
    )
    shortfall["Robust"].append(
        100 * np.mean(np.maximum(0, target - np.array(ef_rob[l]))/per_ha)
    )
    shortfall["Stochastic"].append(
        100 * np.mean(np.maximum(0, target - np.array(ef_sto[l]))/per_ha)
    )

df_shortfall = pd.DataFrame(shortfall, index=[f"P{l}" for l in periods])

# plot heatmap
plt.figure(figsize=(7, 4))
sns.heatmap(
    df_shortfall,
    annot=True,
    fmt=".2f",
    cmap="Reds",
    cbar_kws={"label": "Expected shortfall (%)"}
)


#plt.xlabel("Formulation")
plt.ylabel("Period")
plt.title("Expected harvest shortfall relative to target (%)")
plt.tight_layout()

plt.savefig(
    path_out+"Periodic_expected_shortfall_"+TYPE+".png",
    bbox_inches="tight"
)

plt.show()
